In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import tensorflow_hub as hub

from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
data_dir = '../input/chest-xray-pneumonia/chest_xray'

test_dir = os.path.join(data_dir, 'test/')
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'val/')

model_file = os.path.join('../input/model-7', 'model.h5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42,
    classes=['NORMAL', 'PNEUMONIA']
)

train_generator = test_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42,
    classes=['NORMAL', 'PNEUMONIA']
)

valid_generator = test_datagen.flow_from_directory(
    directory=valid_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42,
    classes=['NORMAL', 'PNEUMONIA']
)

In [ ]:
model = tf.keras.models.load_model(
    model_file,
    custom_objects={
        'KerasLayer': hub.KerasLayer
    }
)

In [ ]:
loss, accuracy = model.evaluate(valid_generator)

print('\nLoss: {:,.3f}'.format(loss))
print('Accuracy: {:.3%}'.format(accuracy))

In [ ]:
loss, accuracy = model.evaluate(train_generator)

print('\nLoss: {:,.3f}'.format(loss))
print('Accuracy: {:.3%}'.format(accuracy))

In [ ]:
loss, accuracy = model.evaluate(test_generator)

print('\nLoss: {:,.3f}'.format(loss))
print('Accuracy: {:.3%}'.format(accuracy))

---
---
---

In [ ]:
batch_size = 32
num_of_train_samples = 5216

In [ ]:
Y_pred = model.predict_generator(train_generator, num_of_train_samples//batch_size+1, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(train_generator.classes, y_pred))

In [ ]:
print('Classification Report')
target_names = ['Normal', 'Pneumonia']
print(classification_report(train_generator.classes, y_pred, target_names=target_names))